# Bert Experiments

In [111]:
import torch
from pytorch_transformers import BertTokenizer, BertModel, BertForMaskedLM, BertConfig
import logging
logging.basicConfig(level=logging.INFO)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def batch_to_idx_tensor(text):
    assert isinstance(text, list), 'Must input a list of strings!'
    str_tokens = [tokenizer.tokenize('[CLS] ' + t + ' [SEP]') for t in text]
    indexed_tokens = [tokenizer.convert_tokens_to_ids(t) for t in str_tokens]
    seq_lens = torch.LongTensor(list(map(len, indexed_tokens)))
    seq_tensor = torch.zeros((len(indexed_tokens), seq_lens.max()), dtype=torch.long)
    attn_mask_tensor = torch.zeros((len(indexed_tokens), seq_lens.max()), dtype=torch.long)
    pad_idx = tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0]
    seq_tensor.fill_(pad_idx)

    for idx, (seq, seqlen) in enumerate(zip(indexed_tokens, seq_lens)):
        seq_tensor[idx, :seqlen] = torch.LongTensor(seq)
        attn_mask_tensor[idx, :seqlen] = 1

    scrm_seq_lens, scrm_idxs = seq_lens.sort(0, descending=True)
    scrm_seq_tensor = seq_tensor[scrm_idxs]
    scrm_str_tokens = [str_tokens[i.item()] for i in scrm_idxs]
    scrm_attn_mask = attn_mask_tensor[scrm_idxs]

    return scrm_seq_tensor, scrm_str_tokens, scrm_attn_mask, scrm_idxs

def str_to_idx_tensor(text, masked_words=None):
    assert isinstance(text, str), 'Must input a string!'
    # tokens_tensor = torch.tensor(indexed_tokens)
    text = '[CLS] ' + text + ' [SEP]'
    tokenized_text = tokenizer.tokenize(text)
    print(tokenized_text)

    if masked_words is not None:
        for i, tok in enumerate(tokenized_text):
            if tok in masked_words:
                tokenized_text[i] = '[MASK]'

    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])

    return tokens_tensor

INFO:pytorch_transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/hansonlu/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [7]:
model = BertModel.from_pretrained('bert-base-uncased', output_attentions=True)

INFO:pytorch_transformers.modeling_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/hansonlu/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
INFO:pytorch_transformers.modeling_utils:Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": true,
  "output_hidden_states": false,
  "torchscript": false,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

INFO:pytorch_transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-pytorch_model.bin not f

  6%|▌         | 26711040/440473133 [00:27<09:34, 719909.75B/s]

  6%|▌         | 26924032/440473133 [00:27<07:45, 887534.95B/s]

  6%|▌         | 27104256/440473133 [00:27<06:37, 1039831.39B/s]

  6%|▌         | 27379712/440473133 [00:28<05:23, 1278581.43B/s]

  6%|▋         | 27595776/440473133 [00:28<04:43, 1454059.72B/s]

  6%|▋         | 27825152/440473133 [00:28<04:14, 1620554.98B/s]

  6%|▋         | 28169216/440473133 [00:28<03:34, 1920712.47B/s]

  6%|▋         | 28419072/440473133 [00:29<15:24, 445542.37B/s] 

  6%|▋         | 28599296/440473133 [00:30<13:18, 515642.52B/s]

  7%|▋         | 28873728/440473133 [00:30<10:17, 666041.81B/s]

  7%|▋         | 29152256/440473133 [00:30<07:58, 859178.15B/s]

  7%|▋         | 29430784/440473133 [00:30<06:22, 1075479.44B/s]

  7%|▋         | 29742080/440473133 [00:30<05:10, 1323355.99B/s]

  7%|▋         | 30069760/440473133 [00:30<04:17, 1593191.79B/s]

  7%|▋         | 30327808/440473133 [00:31<10:50, 630110.76B/s] 

  7%|▋         

 12%|█▏        | 51533824/440473133 [00:54<13:41, 473667.90B/s]

 12%|█▏        | 51745792/440473133 [00:54<10:50, 597785.01B/s]

 12%|█▏        | 51975168/440473133 [00:54<08:39, 747710.25B/s]

 12%|█▏        | 52155392/440473133 [00:54<07:09, 903215.60B/s]

 12%|█▏        | 52352000/440473133 [00:54<06:05, 1062069.08B/s]

 12%|█▏        | 52532224/440473133 [00:54<05:20, 1210706.33B/s]

 12%|█▏        | 52728832/440473133 [00:54<04:43, 1367017.86B/s]

 12%|█▏        | 52903936/440473133 [00:54<04:39, 1386182.65B/s]

 12%|█▏        | 53138432/440473133 [00:54<04:25, 1460372.35B/s]

 12%|█▏        | 53304320/440473133 [00:55<12:28, 517440.41B/s] 

 12%|█▏        | 53427200/440473133 [00:56<13:09, 490176.03B/s]

 12%|█▏        | 53597184/440473133 [00:56<10:52, 592757.47B/s]

 12%|█▏        | 53777408/440473133 [00:56<09:00, 715239.35B/s]

 12%|█▏        | 53974016/440473133 [00:56<07:37, 844869.66B/s]

 12%|█▏        | 54154240/440473133 [00:56<06:25, 1003180.27B/s]

 12%|█▏        | 5

 17%|█▋        | 73501696/440473133 [01:16<10:06, 605343.90B/s]

 17%|█▋        | 73651200/440473133 [01:16<08:38, 707050.42B/s]

 17%|█▋        | 73928704/440473133 [01:17<06:42, 910618.09B/s]

 17%|█▋        | 74109952/440473133 [01:17<05:44, 1063585.34B/s]

 17%|█▋        | 74306560/440473133 [01:17<05:00, 1220073.80B/s]

 17%|█▋        | 74617856/440473133 [01:17<04:05, 1489158.27B/s]

 17%|█▋        | 74880000/440473133 [01:17<03:36, 1687865.48B/s]

 17%|█▋        | 75191296/440473133 [01:17<03:09, 1928253.29B/s]

 17%|█▋        | 75435008/440473133 [01:18<11:08, 545808.92B/s] 

 17%|█▋        | 75748352/440473133 [01:18<08:39, 702447.30B/s]

 17%|█▋        | 76108800/440473133 [01:19<06:47, 894717.63B/s]

 17%|█▋        | 76314624/440473133 [01:19<06:14, 972857.87B/s]

 17%|█▋        | 76567552/440473133 [01:19<05:07, 1185052.93B/s]

 17%|█▋        | 76780544/440473133 [01:19<04:29, 1351586.37B/s]

 17%|█▋        | 77009920/440473133 [01:19<03:59, 1518711.16B/s]

 18%|█▊        |

 23%|██▎       | 102552576/440473133 [01:46<03:45, 1501481.87B/s]

 23%|██▎       | 102913024/440473133 [01:46<03:11, 1764288.76B/s]

 23%|██▎       | 103195648/440473133 [01:48<11:07, 505091.44B/s] 

 24%|██▎       | 103535616/440473133 [01:48<08:30, 659723.68B/s]

 24%|██▎       | 103879680/440473133 [01:48<06:26, 870411.95B/s]

 24%|██▎       | 104174592/440473133 [01:48<05:06, 1095731.54B/s]

 24%|██▎       | 104496128/440473133 [01:48<04:05, 1365834.92B/s]

 24%|██▍       | 104863744/440473133 [01:48<03:19, 1682850.06B/s]

 24%|██▍       | 105168896/440473133 [01:49<08:30, 656858.35B/s] 

 24%|██▍       | 105452544/440473133 [01:50<06:41, 834796.50B/s]

 24%|██▍       | 105747456/440473133 [01:50<05:16, 1056599.56B/s]

 24%|██▍       | 106107904/440473133 [01:50<04:09, 1339380.24B/s]

 24%|██▍       | 106468352/440473133 [01:50<03:29, 1596026.40B/s]

 24%|██▍       | 106779648/440473133 [01:50<02:58, 1868957.54B/s]

 24%|██▍       | 107068416/440473133 [01:52<12:12, 455403.20B/s] 

...

 79%|███████▉  | 349590528/440473133 [06:41<02:18, 658310.05B/s]

 79%|███████▉  | 349787136/440473133 [06:41<01:53, 796967.52B/s]

 79%|███████▉  | 349948928/440473133 [06:41<01:36, 940046.04B/s]

 79%|███████▉  | 350131200/440473133 [06:41<01:25, 1061595.30B/s]

 80%|███████▉  | 350269440/440473133 [06:41<01:20, 1114368.21B/s]

 80%|███████▉  | 350524416/440473133 [06:41<01:11, 1263846.75B/s]

 80%|███████▉  | 350737408/440473133 [06:42<01:02, 1436755.95B/s]

 80%|███████▉  | 350966784/440473133 [06:42<00:56, 1582921.65B/s]

 80%|███████▉  | 351212544/440473133 [06:42<00:51, 1742650.48B/s]

 80%|███████▉  | 351407104/440473133 [06:43<02:42, 547818.87B/s] 

 80%|███████▉  | 351549440/440473133 [06:43<02:20, 633857.11B/s]

 80%|███████▉  | 351720448/440473133 [06:43<02:01, 731295.77B/s]

 80%|███████▉  | 351917056/440473133 [06:43<01:38, 900904.02B/s]

 80%|███████▉  | 352130048/440473133 [06:43<01:25, 1034568.21B/s]

 80%|███████▉  | 352326656/440473133 [06:43<01:14, 1188154.85B/s]

 

 83%|████████▎ | 365990912/440473133 [06:58<01:11, 1040583.52B/s]

 83%|████████▎ | 366105600/440473133 [06:58<01:18, 948936.74B/s] 

 83%|████████▎ | 366236672/440473133 [06:59<01:19, 928102.09B/s]

 83%|████████▎ | 366400512/440473133 [06:59<01:14, 990297.78B/s]

 83%|████████▎ | 366564352/440473133 [06:59<01:10, 1043046.62B/s]

 83%|████████▎ | 366728192/440473133 [06:59<01:07, 1085019.37B/s]

 83%|████████▎ | 366892032/440473133 [06:59<01:04, 1132915.88B/s]

 83%|████████▎ | 367007744/440473133 [06:59<01:05, 1122808.28B/s]

 83%|████████▎ | 367154176/440473133 [06:59<01:01, 1193288.43B/s]

 83%|████████▎ | 367276032/440473133 [06:59<01:02, 1176999.29B/s]

 83%|████████▎ | 367432704/440473133 [07:00<00:57, 1264639.55B/s]

 83%|████████▎ | 367562752/440473133 [07:00<00:59, 1216290.14B/s]

 83%|████████▎ | 367711232/440473133 [07:00<00:58, 1249351.54B/s]

 84%|████████▎ | 367842304/440473133 [07:00<00:57, 1261103.80B/s]

 84%|████████▎ | 367989760/440473133 [07:00<00:55, 1317584.81B/s

 88%|████████▊ | 385799168/440473133 [07:19<01:54, 477736.41B/s]

 88%|████████▊ | 385898496/440473133 [07:19<01:43, 525127.13B/s]

 88%|████████▊ | 386126848/440473133 [07:19<01:21, 669295.88B/s]

 88%|████████▊ | 386323456/440473133 [07:19<01:05, 831575.62B/s]

 88%|████████▊ | 386552832/440473133 [07:19<00:53, 1006529.45B/s]

 88%|████████▊ | 386749440/440473133 [07:20<00:45, 1176103.11B/s]

 88%|████████▊ | 386978816/440473133 [07:20<00:38, 1372912.87B/s]

 88%|████████▊ | 387175424/440473133 [07:20<00:35, 1508280.50B/s]

 88%|████████▊ | 387421184/440473133 [07:20<00:31, 1702018.20B/s]

 88%|████████▊ | 387627008/440473133 [07:20<00:37, 1394164.09B/s]

 88%|████████▊ | 387800064/440473133 [07:20<00:39, 1348450.71B/s]

 88%|████████▊ | 387978240/440473133 [07:20<00:38, 1371530.17B/s]

 88%|████████▊ | 388158464/440473133 [07:20<00:35, 1475782.65B/s]

 88%|████████▊ | 388322304/440473133 [07:21<00:35, 1477189.91B/s]

 88%|████████▊ | 388486144/440473133 [07:21<00:34, 1506413.24B/s]


 93%|█████████▎| 410440704/440473133 [07:44<00:30, 972949.33B/s]

 93%|█████████▎| 410647552/440473133 [07:44<00:25, 1156706.11B/s]

 93%|█████████▎| 410883072/440473133 [07:44<00:22, 1330322.85B/s]

 93%|█████████▎| 411112448/440473133 [07:45<00:19, 1496507.88B/s]

 93%|█████████▎| 411390976/440473133 [07:45<00:16, 1733720.21B/s]

 93%|█████████▎| 411606016/440473133 [07:46<00:54, 526887.77B/s] 

 93%|█████████▎| 411800576/440473133 [07:46<00:44, 647775.85B/s]

 94%|█████████▎| 412079104/440473133 [07:46<00:34, 822741.97B/s]

 94%|█████████▎| 412275712/440473133 [07:46<00:28, 981358.98B/s]

 94%|█████████▎| 412587008/440473133 [07:46<00:22, 1222626.03B/s]

 94%|█████████▎| 412800000/440473133 [07:46<00:20, 1373819.81B/s]

 94%|█████████▍| 413176832/440473133 [07:46<00:16, 1673670.78B/s]

 94%|█████████▍| 413422592/440473133 [07:47<00:45, 595513.37B/s] 

 94%|█████████▍| 413601792/440473133 [07:48<00:41, 650237.22B/s]

 94%|█████████▍| 413897728/440473133 [07:48<00:32, 810336.59B/s]

 

 98%|█████████▊| 431019008/440473133 [08:06<00:16, 569065.39B/s]

 98%|█████████▊| 431199232/440473133 [08:06<00:13, 686948.18B/s]

 98%|█████████▊| 431330304/440473133 [08:06<00:11, 799806.74B/s]

 98%|█████████▊| 431510528/440473133 [08:06<00:09, 941291.67B/s]

 98%|█████████▊| 431690752/440473133 [08:06<00:08, 1078476.22B/s]

 98%|█████████▊| 431870976/440473133 [08:07<00:07, 1111402.82B/s]

 98%|█████████▊| 432133120/440473133 [08:07<00:06, 1272421.91B/s]

 98%|█████████▊| 432411648/440473133 [08:07<00:05, 1517391.14B/s]

 98%|█████████▊| 432601088/440473133 [08:08<00:18, 434348.46B/s] 

 98%|█████████▊| 432739328/440473133 [08:08<00:18, 424516.11B/s]

 98%|█████████▊| 432935936/440473133 [08:09<00:13, 542248.13B/s]

 98%|█████████▊| 433156096/440473133 [08:09<00:10, 700677.44B/s]

 98%|█████████▊| 433312768/440473133 [08:09<00:08, 825610.15B/s]

 98%|█████████▊| 433492992/440473133 [08:09<00:07, 984949.23B/s]

 98%|█████████▊| 433705984/440473133 [08:09<00:05, 1145552.23B/s]

 99%

In [8]:
model.eval()



In [90]:
text = 'burma has put five cities on a security alert after religious unrest involving buddhists and moslems in the northern city of mandalay , an informed source said wednesday.' 
text1 = 'police arrested five anti-nuclear protesters friday after they sought to disrupt loading of a french antarctic research and supply vessel , a spokesman for the protesters said .'
text2 = 'turkmen president gurbanguly berdymukhammedov will begin a two-day visit to russia , his country \'s main energy partner , on monday for trade talks , the kremlin press office said .'
text3 = 'israel \'s new government barred yasser arafat from flying to the west bank to meet with former prime minister shimon peres on thursday , a move palestinian officials said violated the israel-plo peace accords .'
toks = str_to_idx_tensor(text)

with torch.no_grad():
    outputs = model(toks)

['[CLS]', 'burma', 'has', 'put', 'five', 'cities', 'on', 'a', 'security', 'alert', 'after', 'religious', 'unrest', 'involving', 'buddhist', '##s', 'and', 'mo', '##sle', '##ms', 'in', 'the', 'northern', 'city', 'of', 'mandal', '##ay', ',', 'an', 'informed', 'source', 'said', 'wednesday', '.', '[SEP]']
tensor([[  101, 22883,  3549,  2343, 19739, 28483,  3070,  5313,  2100,  2022,
         17460, 12274, 15256, 20058,  3527,  2615,  2097,  4088,  1037,  2048,
          1011,  2154,  3942,  2000,  3607,  1010,  2010,  2406,  1005,  1055,
          2364,  2943,  4256,  1010,  2006,  6928,  2005,  3119,  7566,  1010,
          1996,  1047, 28578,  4115,  2811,  2436,  2056,  1012,   102],
        [  101,  3956,  1005,  1055,  2047,  2231, 15605,  8038, 18116, 19027,
         27753,  2013,  3909,  2000,  1996,  2225,  2924,  2000,  3113,  2007,
          2280,  3539,  2704, 11895,  8202, 23976,  2015,  2006,  9432,  1010,
          1037,  2693,  9302,  4584,  2056, 14424,  1996,  3956,  1011, 

In [68]:
# print(outputs[0].shape) 
# (batch_size, sequence_length, hidden_size) 
# Sequence of hidden-states at the output of the last layer of the model.

# print(outputs[1].shape)
# pooler_output, probably not relevant to our usage

# print(len(outputs[2])) 
# attentions of all 12 layers
# print(outputs[2][0].shape) 
# (batch_size, num_heads, sequence_length, sequence_length)
# the third dimension sums up to one
# print(torch.sum(outputs[2][0], dim=3))
tokens = tokenizer.convert_ids_to_tokens(toks.tolist()[0])
cum_attn = []
layers = [0, 8, 9, 10, 11]
for l in layers:
    print('---- Layer', l)
    layer = outputs[2][l]
    summed = layer.sum(dim=2).sum(dim=1).view(-1)
    summed = (summed / summed.sum(dim=0)).tolist()
    cum_attn.append(summed)
    sorted_by_attn = sorted(list(zip(summed, tokens)), key=lambda p: p[0], reverse=True)
    print(sorted_by_attn[:10])
print('---- total')
sum_attn = torch.tensor(cum_attn).sum(dim=0)
sum_attn = (sum_attn / sum_attn.sum(dim=0)).tolist()
sorted_by_attn = sorted(list(zip(sum_attn, tokens)), key=lambda p: p[0], reverse=True)
print(sorted_by_attn[:10])

---- Layer 0
[(0.08032415807247162, '[CLS]'), (0.04123055562376976, 'palestinian'), (0.03168165683746338, 'israel'), (0.0302461925894022, 'thursday'), (0.02916713058948517, 'violated'), (0.02887265384197235, 'minister'), (0.02798117883503437, 'israel'), (0.026993650943040848, '##fat'), (0.025710370391607285, 'accord'), (0.0246791560202837, '[SEP]')]
---- Layer 9
[(0.4811535179615021, '[SEP]'), (0.0437617227435112, '.'), (0.037608399987220764, '[CLS]'), (0.029492225497961044, 'israel'), (0.02293320745229721, 'palestinian'), (0.021103298291563988, '##fat'), (0.020826423540711403, 'barred'), (0.01810401678085327, 'thursday'), (0.018085891380906105, '##sser'), (0.017743797972798347, 'pere')]
---- Layer 10
[(0.3741353750228882, '.'), (0.19038298726081848, ','), (0.059014927595853806, '[CLS]'), (0.050655387341976166, '[SEP]'), (0.02185649797320366, 'palestinian'), (0.020123891532421112, 'pere'), (0.01932804472744465, 'thursday'), (0.019161606207489967, 'israel'), (0.013851759023964405, 'flyi

In [112]:
text = 'burma has put five cities on a security alert after religious unrest involving buddhists and moslems in the northern city of mandalay , an informed source said wednesday.' 
text1 = 'police arrested five anti-nuclear protesters friday after they sought to disrupt loading of a french antarctic research and supply vessel , a spokesman for the protesters said .'
text2 = 'turkmen president gurbanguly berdymukhammedov will begin a two-day visit to russia , his country \'s main energy partner , on monday for trade talks , the kremlin press office said .'
text3 = 'israel \'s new government barred yasser arafat from flying to the west bank to meet with former prime minister shimon peres on thursday , a move palestinian officials said violated the israel-plo peace accords .'
batch = [text, text1, text2, text3]
batch_toks, str_toks, attn_mask, scrm_idxs = batch_to_idx_tensor(batch)

print(batch_toks)
print(attn_mask)
print([len(s) for s in str_toks])

with torch.no_grad():
    outputs = model(batch_toks, attention_mask=attn_mask)

print(outputs[2][0].shape)

['[CLS]', 'burma', 'has', 'put', 'five', 'cities', 'on', 'a', 'security', 'alert', 'after', 'religious', 'unrest', 'involving', 'buddhist', '##s', 'and', 'mo', '##sle', '##ms', 'in', 'the', 'northern', 'city', 'of', 'mandal', '##ay', ',', 'an', 'informed', 'source', 'said', 'wednesday', '.', '[SEP]']
tensor([[  101, 22883,  3549,  2343, 19739, 28483,  3070,  5313,  2100,  2022,
         17460, 12274, 15256, 20058,  3527,  2615,  2097,  4088,  1037,  2048,
          1011,  2154,  3942,  2000,  3607,  1010,  2010,  2406,  1005,  1055,
          2364,  2943,  4256,  1010,  2006,  6928,  2005,  3119,  7566,  1010,
          1996,  1047, 28578,  4115,  2811,  2436,  2056,  1012,   102],
        [  101,  3956,  1005,  1055,  2047,  2231, 15605,  8038, 18116, 19027,
         27753,  2013,  3909,  2000,  1996,  2225,  2924,  2000,  3113,  2007,
          2280,  3539,  2704, 11895,  8202, 23976,  2015,  2006,  9432,  1010,
          1037,  2693,  9302,  4584,  2056, 14424,  1996,  3956,  1011, 

In [138]:
from collections import defaultdict as DD

attended_words = DD(int)

layers = torch.tensor([0, 9, 10, 11])

attn = torch.stack(outputs[2]).index_select(0, layers) # [4, 4, 12, 49, 49]
summed = attn.sum(dim=3).sum(dim=2).sum(dim=0).view(attn.shape[1], attn.shape[4])
summed = summed / summed.sum(dim=1, keepdim=True)
_, topk_idxs = summed.topk(10, sorted=False)
print(topk_idxs)

attended_word_tensor = torch.zeros(tokenizer.vocab_size)

split_batch_toks = batch_toks.split(1, dim=0)
split_topk_idxs = topk_idxs.split(1, dim=0)
print(split_batch_toks)
for idxs, toks in zip(topk_idxs, split_batch_toks):
    attended_toks = toks.view(-1)[idxs.view(-1)]
    attended_word_tensor[attended_toks] += 1
print(attended_word_tensor.nonzero().view(-1))

non_zero_idxs = attended_word_tensor.nonzero().view(-1)
counts = attended_word_tensor[non_zero_idxs].tolist()
toks = tokenizer.convert_ids_to_tokens(non_zero_idxs.tolist())
attended_words.update(zip(toks, counts))
print(attended_words)
print(to)
# offset = (torch.arange(batch_toks.shape[0]) * batch_toks.shape[1]).unsqueeze(1)
# attended_idxs = topk_idxs + offset
# attended_tok_idxs = batch_toks.take(attended_idxs)
# print(attended_tok_idxs)
# attended_word_tensor[attended_tok_idxs] += 1
# print(attended_word_tensor[102])

# summed = (summed / summed.sum(dim=0)).tolist()
# cum_attn.append(summed)
# sorted_by_attn = sorted(list(zip(summed, tokens)), key=lambda p: p[0], reverse=True)
# print(sorted_by_attn[:10])

tensor([[16, 24,  1,  3,  2, 31,  0, 33, 47, 48],
        [25, 10,  6, 28,  1, 32, 45, 44, 29,  0],
        [25, 11,  2, 12, 32,  1, 14,  0, 34, 33],
        [ 1,  8, 28, 17,  6, 31, 30,  7, 18,  0]])
(tensor([[  101, 22883,  3549,  2343, 19739, 28483,  3070,  5313,  2100,  2022,
         17460, 12274, 15256, 20058,  3527,  2615,  2097,  4088,  1037,  2048,
          1011,  2154,  3942,  2000,  3607,  1010,  2010,  2406,  1005,  1055,
          2364,  2943,  4256,  1010,  2006,  6928,  2005,  3119,  7566,  1010,
          1996,  1047, 28578,  4115,  2811,  2436,  2056,  1012,   102]]), tensor([[  101,  3956,  1005,  1055,  2047,  2231, 15605,  8038, 18116, 19027,
         27753,  2013,  3909,  2000,  1996,  2225,  2924,  2000,  3113,  2007,
          2280,  3539,  2704, 11895,  8202, 23976,  2015,  2006,  9432,  1010,
          1037,  2693,  9302,  4584,  2056, 14424,  1996,  3956,  1011, 20228,
          2080,  3521, 15802,  2015,  1012,   102,     0,     0,     0]]), tensor([[  101, 